In [1]:
# Clone git to get the data
!git clone https://github.com/rasyidpurnama/abd2022

Cloning into 'abd2022'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 2), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [2]:
cd abd2022

/content/abd2022


In [6]:
# Importing packages 
import numpy as np
import pandas as pd
from functools import reduce

In [131]:
# Read the data
df = pd.read_csv("mtsamples_20220222.csv").drop_duplicates().dropna().reset_index(drop=True)
df.head()

,medical_speciality,sample_name,description,transcription,keywords
0,Allergy / Immunology,Allergic Rhinitis,A 23-year-old white female presents with compl...,SUBJECTIVE: This 23-year-old white female pres...,"allergy / immunology, allergic rhinitis, aller..."
1,Allergy / Immunology,Allergy Evaluation Consult,"Acute allergic reaction, etiology uncertain, h...",HISTORY: A 34-year-old male presents today sel...,"allergy / immunology, keflex, acute allergic r..."
2,Allergy / Immunology,Asthma in a 5-year-old,Mother states he has been wheezing and coughing.,CHIEF COMPLAINT: This 5-year-old male presents...,"allergy / immunology, breathing treatment, air..."
3,Allergy / Immunology,Chronic Sinusitis,Patient having severe sinusitis about two to t...,HISTORY: I had the pleasure of meeting and eva...,"allergy / immunology, nasal congestion, facial..."
4,Allergy / Immunology,Evaluation of Allergies,"Chronic glossitis, xerostomia, probable enviro...",HISTORY: A 55-year-old female presents self-re...,"allergy / immunology, chronic glossitis, xeros..."


In [433]:
# Create comparison to find the maximum of keywords existance in the transcription
# example: in row 3 sinusitis is the maximum existance in the transcription


# Function to count existance of keyword in the transcription (formula = len(before)-len(after))

# [LOOPING FUNCTION]
def looping_wordcount(keyword, transcript):
    return (len(transcript)-len(transcript.replace(keyword,"")))//(len(keyword)+0.0001)

print("=== Given List of Keywords ===")
list_of_transcription = df.transcription.tolist()
list_of_keywords = df.keywords.tolist()
print("len of list_of_transcription:",len(list_of_transcription))
print("len of list_of_keywords:",len(list_of_keywords))
print()

=== Given List of Keywords ===
len of list_of_transcription: 4997
len of list_of_keywords: 4997



In [440]:
%%time
print("=== Using For Loop ===")
list_of_most_number_of_keywords = []
for transcript, keywords in zip(list_of_transcription,list_of_keywords):
    count_list = []
    for keyword in keywords.split(', '):
        count_list.append(looping_wordcount(keyword, transcript))
    counter = max(count_list)
    list_of_most_number_of_keywords.append(counter)
df['looping_count'] = list_of_most_number_of_keywords

=== Using For Loop ===
CPU times: user 508 ms, sys: 1.8 ms, total: 510 ms
Wall time: 615 ms


In [441]:
%%time
print("=== Using Map Reduce ===")
map_split = lambda doc,key: (len(doc)-len(doc.replace(key,"")))//(len(key)+0.0001)
map_collect = lambda i: map(map_split, [list_of_transcription[i] for k in range(len(list_of_keywords[i].split(", ")))], list_of_keywords[i].split(", "))
map_collects = map(map_collect, df.index)
reduce_find_max = lambda a,b: max(a,b)
map_get_all_most = lambda x: reduce(reduce_find_max, x)
map_most_keywords = map(map_get_all_most, map_collects)
df['mapreduce_count'] = list(map_most_keywords)

=== Using Map Reduce ===
CPU times: user 489 ms, sys: 926 µs, total: 490 ms
Wall time: 561 ms


In [442]:
%%time
print("=== Using Numpy Pandas ===")
df['numpy_count'] = df.apply(lambda x: max([looping_wordcount(keyword,x['transcription']) for keyword in x['keywords'].split(', ')]), axis=1)

=== Using Numpy Pandas ===
CPU times: user 799 ms, sys: 1.87 ms, total: 800 ms
Wall time: 805 ms


In [432]:
df.head()

,medical_speciality,sample_name,description,transcription,keywords,looping_count,mapreduce_count,numpy_count
0,Allergy / Immunology,Allergic Rhinitis,A 23-year-old white female presents with compl...,SUBJECTIVE: This 23-year-old white female pres...,"allergy / immunology, allergic rhinitis, aller...",2.0,2.0,2.0
1,Allergy / Immunology,Allergy Evaluation Consult,"Acute allergic reaction, etiology uncertain, h...",HISTORY: A 34-year-old male presents today sel...,"allergy / immunology, keflex, acute allergic r...",5.0,5.0,5.0
2,Allergy / Immunology,Asthma in a 5-year-old,Mother states he has been wheezing and coughing.,CHIEF COMPLAINT: This 5-year-old male presents...,"allergy / immunology, breathing treatment, air...",1.0,1.0,1.0
3,Allergy / Immunology,Chronic Sinusitis,Patient having severe sinusitis about two to t...,HISTORY: I had the pleasure of meeting and eva...,"allergy / immunology, nasal congestion, facial...",14.0,14.0,14.0
4,Allergy / Immunology,Evaluation of Allergies,"Chronic glossitis, xerostomia, probable enviro...",HISTORY: A 55-year-old female presents self-re...,"allergy / immunology, chronic glossitis, xeros...",2.0,2.0,2.0


In [444]:
# jumlah baris x 20
df_big = pd.concat([df]*20).reset_index(drop=True)

In [446]:
print("=== Given List of Keywords ===")
list_of_transcription = df_big.transcription.tolist()
list_of_keywords = df_big.keywords.tolist()
print("len of list_of_transcription:",len(list_of_transcription))
print("len of list_of_keywords:",len(list_of_keywords))
print()

=== Given List of Keywords ===
len of list_of_transcription: 99940
len of list_of_keywords: 99940



In [459]:
%%time
print("=== Using For Loop ===")
list_of_most_number_of_keywords = []
for transcript, keywords in zip(list_of_transcription,list_of_keywords):
    count_list = []
    for keyword in keywords.split(', '):
        count_list.append(looping_wordcount(keyword, transcript))
    counter = max(count_list)
    list_of_most_number_of_keywords.append(counter)
df_big['looping_count'] = list_of_most_number_of_keywords

=== Using For Loop ===
CPU times: user 7.69 s, sys: 32.4 ms, total: 7.72 s
Wall time: 7.76 s


In [460]:
%%time
print("=== Using Map Reduce ===")
map_split = lambda doc,key: (len(doc)-len(doc.replace(key,"")))//(len(key)+0.0001)
map_collect = lambda i: map(map_split, [list_of_transcription[i] for k in range(len(list_of_keywords[i].split(", ")))], list_of_keywords[i].split(", "))
map_collects = map(map_collect, df_big.index)
reduce_find_max = lambda a,b: max(a,b)
map_get_all_most = lambda x: reduce(reduce_find_max, x)
map_most_keywords = map(map_get_all_most, map_collects)
df_big['mapreduce_count'] = list(map_most_keywords)

=== Using Map Reduce ===
CPU times: user 8.18 s, sys: 44.6 ms, total: 8.23 s
Wall time: 8.31 s


In [457]:
%%time
print("=== Using Numpy Pandas ===")
df_big['numpy_count'] = df_big.apply(lambda x: max([looping_wordcount(keyword,x['transcription']) for keyword in x['keywords'].split(', ')]), axis=1)

=== Using Numpy Pandas ===
CPU times: user 15.3 s, sys: 86.5 ms, total: 15.4 s
Wall time: 15.5 s


In [458]:
df_big.head()

,medical_speciality,sample_name,description,transcription,keywords,looping_count,mapreduce_count,numpy_count
0,Allergy / Immunology,Allergic Rhinitis,A 23-year-old white female presents with compl...,SUBJECTIVE: This 23-year-old white female pres...,"allergy / immunology, allergic rhinitis, aller...",2.0,2.0,2.0
1,Allergy / Immunology,Allergy Evaluation Consult,"Acute allergic reaction, etiology uncertain, h...",HISTORY: A 34-year-old male presents today sel...,"allergy / immunology, keflex, acute allergic r...",5.0,5.0,5.0
2,Allergy / Immunology,Asthma in a 5-year-old,Mother states he has been wheezing and coughing.,CHIEF COMPLAINT: This 5-year-old male presents...,"allergy / immunology, breathing treatment, air...",1.0,1.0,1.0
3,Allergy / Immunology,Chronic Sinusitis,Patient having severe sinusitis about two to t...,HISTORY: I had the pleasure of meeting and eva...,"allergy / immunology, nasal congestion, facial...",14.0,14.0,14.0
4,Allergy / Immunology,Evaluation of Allergies,"Chronic glossitis, xerostomia, probable enviro...",HISTORY: A 55-year-old female presents self-re...,"allergy / immunology, chronic glossitis, xeros...",2.0,2.0,2.0


In [461]:
# jumlah data di kolom transcription x 20
df_big['transcription'] = df_big['transcription']*20

In [462]:
print("=== Given List of Keywords ===")
list_of_transcription = df_big.transcription.tolist()
list_of_keywords = df_big.keywords.tolist()
print("len of list_of_transcription:",len(list_of_transcription))
print("len of list_of_keywords:",len(list_of_keywords))
print()

=== Given List of Keywords ===
len of list_of_transcription: 99940
len of list_of_keywords: 99940



In [466]:
%%time
print("=== Using For Loop ===")
list_of_most_number_of_keywords = []
for transcript, keywords in zip(list_of_transcription,list_of_keywords):
    count_list = []
    for keyword in keywords.split(', '):
        count_list.append(looping_wordcount(keyword, transcript))
    counter = max(count_list)
    list_of_most_number_of_keywords.append(counter)
df_big['looping_count'] = list_of_most_number_of_keywords

=== Using For Loop ===
CPU times: user 1min 5s, sys: 1.28 s, total: 1min 6s
Wall time: 1min 13s


In [467]:
%%time
print("=== Using Map Reduce ===")
map_split = lambda doc,key: (len(doc)-len(doc.replace(key,"")))//(len(key)+0.0001)
map_collect = lambda i: map(map_split, [list_of_transcription[i] for k in range(len(list_of_keywords[i].split(", ")))], list_of_keywords[i].split(", "))
map_collects = map(map_collect, df_big.index)
reduce_find_max = lambda a,b: max(a,b)
map_get_all_most = lambda x: reduce(reduce_find_max, x)
map_most_keywords = map(map_get_all_most, map_collects)
df_big['mapreduce_count'] = list(map_most_keywords)

=== Using Map Reduce ===
CPU times: user 1min 1s, sys: 545 ms, total: 1min 2s
Wall time: 1min 2s


In [468]:
%%time
print("=== Using Numpy Pandas ===")
df_big['numpy_count'] = df_big.apply(lambda x: max([looping_wordcount(keyword,x['transcription']) for keyword in x['keywords'].split(', ')]), axis=1)

=== Using Numpy Pandas ===
CPU times: user 1min 12s, sys: 490 ms, total: 1min 13s
Wall time: 1min 13s


Kesimlpulannya, dengan menggunakan map reduce bisa menjadi lebih cepat daripada numpy ataupun looping jika ukuran datanya semakin besar dan padat